# Downloading and uploading a project
In this notebook we'll see how to download all available data for a project (media, annotations, predictions, models and the configuration) to our local machine. Once the project data is downloaded, we have effectively made a back-up of our project from the platform to the local disk. 

At the end of the notebook, we'll use the downloaded data to re-create the project on the platform again, and upload all media and annotations to it. This could for example be used to migrate a project to a different server, or to make sure the project data is safe if a server outage is expected for whatever reason.

In [ ]:
# As usual we'll connnect to the platform first, using the credentials from the .env file. We'll also create a ProjectManager for the cluster

from dotenv import dotenv_values
from sc_api_tools import SCRESTClient
from sc_api_tools.rest_managers import ProjectManager

env_variables = dotenv_values(dotenv_path=".env")

if not env_variables:
    print("Unable to load login details from .env file, please make sure the file exists at the root of the notebooks directory.")

client = SCRESTClient(
    host=env_variables.get('HOST'),
    username=env_variables.get('USERNAME'),
    password=env_variables.get('PASSWORD')
)

project_manager = ProjectManager(session=client.session, workspace_id=client.workspace_id)

### Select project to download from the project list
As usual, let's list the available projects and pick one to download. You can change the selected project by changing the `PROJECT_NAME` variable, two cells down.

In [ ]:
projects = project_manager.list_projects()

Specify the name of the project to download. We'll use the `COCO anomalous animal demo` project created in notebook [004](004_create_pipeline_project_from_dataset.ipynb).

In [ ]:
PROJECT_NAME = "COCO anomalous animal demo"

## Project download
Now, let's do the project download itself. The `SCRESTClient` provides a method `download_project()` to do so. It takes the following arguments:

- `project_name`: Name of the project to download
- `target_folder`: Path of the folder to download to. If left empty, a folder named `project_name` will be created in the current directory
- `include_predictions`: True to download predictions for all media, False to not download any predictions
- `include_active_models`: True to download the active models for all tasks in the project, False to not download any models
- `include_deployment`: True to create and download a deployment for the project, False to not create any deployment

Note that downloading may take some time especially for large projects, so please be patient. The method will print regular updates related to the download progress.

In [ ]:
import os

project = client.download_project(
    project_name=PROJECT_NAME, 
    target_folder=os.path.join("projects", PROJECT_NAME), 
    include_predictions=False, 
    include_active_models=False,
    include_deployment=False
)

That's all there is to it! Now, you should have a folder `projects` showing up in the current directory. Inside it should be another folder named `{PROJECT_NAME}`, which should contain all a file `project.json` holding the project details, as well as all media and annotations in the project and a file `configuration.json` that contains the full project configuration. 

In addition, the `download_project` method can also create a deployment for the project (see notebook [008 deploy_project](008_deploy_project.ipynb) for more details on deployments), if the parameter `include_deployment` is set to True. In that case you should see a folder called `deployment` in the project directory as well.

Note that in this case predictions, models and the deployment are not included because we have set `include_predictions=False`, `include_active_models=False` and `include_deployment=False`.

## Project upload
Now that the project is downloaded, we can use it to create a new project on the platform. Once the project is created, we can upload the media, annotations and configuration that were downloaded to the project folder to it. The `client` provides an `upload_project()` method to do all that, which takes three parameters:
- `target_folder`: Path to the folder containing the project data to upload
- `project_name`: Optional name to assign to the new project on the platform. If left unspecified, the project name will correspond to the downloaded project name
- `enable_auto_train`: True to turn on auto training for all tasks in the project, once the media and annotation upload is complete. False to leave auto training turned off.

In [ ]:
uploaded_project = client.upload_project(
    target_folder=os.path.join("projects", PROJECT_NAME),
    project_name=PROJECT_NAME + ' upload',
    enable_auto_train=False
)

Done! The uploaded project should now show up in your workspace. Note that it is of course also possible to upload the project to a different server, simply by setting up a new client to that server and calling the `upload_project()` method from that client.